In [1]:
import polars as pl
import pandas as pd

In [10]:
csv_path = "Resources/heart_attack_prediction_dataset.csv"
heart_df = pl.read_csv(csv_path)

In [26]:
heart_df.head()

Patient ID,Age,Sex,Cholesterol,Blood Pressure,Heart Rate,Diabetes,Family History,Smoking,Obesity,Alcohol Consumption,Exercise Hours Per Week,Diet,Previous Heart Problems,Medication Use,Stress Level,Sedentary Hours Per Day,Income,BMI,Triglycerides,Physical Activity Days Per Week,Sleep Hours Per Day,Country,Continent,Hemisphere,Heart Attack Risk
str,i64,str,i64,str,i64,i64,i64,i64,i64,i64,f64,str,i64,i64,i64,f64,i64,f64,i64,i64,i64,str,str,str,i64
"""BMW7812""",67,"""Male""",208,"""158/88""",72,0,0,1,0,0,4.168189,"""Average""",0,0,9,6.615001,261404,31.251233,286,0,6,"""Argentina""","""South America""","""Southern Hemis…",0
"""CZE1114""",21,"""Male""",389,"""165/93""",98,1,1,1,1,1,1.813242,"""Unhealthy""",1,0,1,4.963459,285768,27.194973,235,1,7,"""Canada""","""North America""","""Northern Hemis…",0
"""BNI9906""",21,"""Female""",324,"""174/99""",72,1,0,0,0,0,2.078353,"""Healthy""",1,1,9,9.463426,235282,28.176571,587,4,4,"""France""","""Europe""","""Northern Hemis…",0
"""JLN3497""",84,"""Male""",383,"""163/100""",73,1,1,1,0,1,9.82813,"""Average""",1,0,9,7.648981,125640,36.464704,378,3,4,"""Canada""","""North America""","""Northern Hemis…",0
"""GFO8847""",66,"""Male""",318,"""91/88""",93,1,1,1,1,0,5.804299,"""Unhealthy""",1,0,6,1.514821,160555,21.809144,231,1,5,"""Thailand""","""Asia""","""Northern Hemis…",0


In [31]:
heart_df = heart_df.drop(['Obesity', 'Exercise Hours Per Week', 'Diet', 'Medication Use', 'Sedentary Hours Per Day', 'Hemisphere', 'Heart Attack Risk'])

In [36]:
heart_df = heart_df[['Patient ID', 'Age', 'Sex', 'Income', 'Country', 'Continent', 'BMI', 'Blood Pressure', 'Heart Rate', 'Cholesterol', 'Triglycerides', 'Diabetes', 'Family History', 'Previous Heart Problems', 'Alcohol Consumption', 'Smoking', 'Stress Level', 'Sleep Hours Per Day', 'Physical Activity Days Per Week']]

In [53]:
heart_df.head()

Patient ID,Age,Sex,Income,Country,Continent,BMI,Blood Pressure,Heart Rate,Cholesterol,Triglycerides,Diabetes,Family History,Previous Heart Problems,Alcohol Consumption,Smoking,Stress Level,Sleep Hours Per Day,Physical Activity Days Per Week
str,i64,str,i64,str,str,f64,str,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64
"""BMW7812""",67,"""Male""",261404,"""Argentina""","""South America""",31.251233,"""158/88""",72,208,286,0,0,0,0,1,9,6,0
"""CZE1114""",21,"""Male""",285768,"""Canada""","""North America""",27.194973,"""165/93""",98,389,235,1,1,1,1,1,1,7,1
"""BNI9906""",21,"""Female""",235282,"""France""","""Europe""",28.176571,"""174/99""",72,324,587,1,0,1,0,0,9,4,4
"""JLN3497""",84,"""Male""",125640,"""Canada""","""North America""",36.464704,"""163/100""",73,383,378,1,1,1,1,1,9,4,3
"""GFO8847""",66,"""Male""",160555,"""Thailand""","""Asia""",21.809144,"""91/88""",93,318,231,1,1,1,0,1,6,5,1


In [67]:
heart_df = heart_df.with_columns(pl.col('Diabetes').cast(pl.Boolean).alias('Diabetes'), 
           pl.col('Family History').cast(pl.Boolean).alias('Family History'),
           pl.col('Previous Heart Problems').cast(pl.Boolean).alias('Previous Heart Problems'),
           pl.col('Alcohol Consumption').cast(pl.Boolean).alias('Alcohol Consumption'),
           pl.col('Smoking').cast(pl.Boolean).alias('Smoking')
)

In [68]:
heart_df

Patient ID,Age,Sex,Income,Country,Continent,BMI,Blood Pressure,Heart Rate,Cholesterol,Triglycerides,Diabetes,Family History,Previous Heart Problems,Alcohol Consumption,Smoking,Stress Level,Sleep Hours Per Day,Physical Activity Days Per Week
str,i64,str,i64,str,str,f64,str,i64,i64,i64,bool,bool,bool,bool,bool,i64,i64,i64
"""BMW7812""",67,"""Male""",261404,"""Argentina""","""South America""",31.251233,"""158/88""",72,208,286,false,false,false,false,true,9,6,0
"""CZE1114""",21,"""Male""",285768,"""Canada""","""North America""",27.194973,"""165/93""",98,389,235,true,true,true,true,true,1,7,1
"""BNI9906""",21,"""Female""",235282,"""France""","""Europe""",28.176571,"""174/99""",72,324,587,true,false,true,false,false,9,4,4
"""JLN3497""",84,"""Male""",125640,"""Canada""","""North America""",36.464704,"""163/100""",73,383,378,true,true,true,true,true,9,4,3
"""GFO8847""",66,"""Male""",160555,"""Thailand""","""Asia""",21.809144,"""91/88""",93,318,231,true,true,true,false,true,6,5,1
"""ZOO7941""",54,"""Female""",241339,"""Germany""","""Europe""",20.14684,"""172/86""",48,297,795,true,true,true,true,true,2,10,5
"""WYV0966""",90,"""Male""",190450,"""Canada""","""North America""",28.885811,"""102/73""",84,358,284,false,false,false,true,true,7,10,4
"""XXM0972""",84,"""Male""",122093,"""Japan""","""Asia""",22.221862,"""131/68""",107,220,370,false,false,false,true,true,4,7,6
"""XCQ5937""",20,"""Male""",25086,"""Brazil""","""South America""",35.809901,"""144/105""",68,145,790,true,false,false,false,true,5,4,7


In [ ]:
# Convert Polars DataFrame to Pandas DataFrame
pd_df = df.to_pandas()
# Convert Pandas DataFrame to SQL
sql_table_name = ‘your_table_name’
sql_statement = pd_df.to_sql(sql_table_name, con=‘sqlite:///:memory:’, index=False, if_exists=‘replace’)
# Print the SQL statement
print(sql_statement)